In [1]:
import dataloader_test as dlt


train_files = [f'../data/modified_ndws/train_conus_west_ndws_0{i:02}.tfrecord' for i in range(39)]
dataset = dlt.MultiTFRecordDataset(train_files)
loader = dlt.DataLoader(dataset, batch_size=32)


In [2]:
from sympy.physics import units


length = units.Dimension("length")
population = units.Dimension("population")
time = units.Dimension("time")
velocity = length/time
mass = units.Dimension("mass")
energy = mass * velocity**2
unit = units.Dimension(1)
density = 1/length**2
temperature = units.Dimension("temperature")

units_ = {
    'elevation': length,
    'population': population*density,
    'chili': unit,
    'pdsi': unit,
    'NDVI': unit,
    'viirs_FireMask': unit,
    'viirs_PrevFireMask': unit,
    'fuel1': unit,
    'fuel2': unit,
    'fuel3': unit,
    'water': unit,
    'impervious': unit,
    'erc': energy*density,  # energy release component
    'sph': unit,  # humidity
    'th': unit,  # wind direction
    'pr': unit, # precipitation
    'vs': velocity,  # wind speed
    'bi': unit,   # burning index
    'tmmx': temperature,
    'tmmn': temperature,
    'boltzmann': energy/temperature
}


dataset = dlt.NondimFireDataset(train_files, units_, positive=["elevation", "population", "vs"], constants={"boltzmann": 1})
loader = dlt.DataLoader(dataset, batch_size=32)

In [3]:
import numpy as np

found = None
for batch in dataset:
    if np.isnan(batch).sum() > 0:
        found = batch
        break
        
thing = None
print(found.shape)
for i, var in enumerate(found):
    if np.isnan(var).sum() > 0:
        thing = dataset.cols[i]

AttributeError: 'NoneType' object has no attribute 'shape'

In [8]:
for batch in loader:
    print(batch.shape)

torch.Size([32, 16, 4096])
torch.Size([7, 16, 4096])


In [20]:
np.where((dataset.nondims < 0).sum(axis=0))

(array([12, 18]),)

In [15]:
0.**0

1.0

In [15]:
from data import nondim
from sympy.physics import units
import numpy as np

x = [{"force": np.array([[1, 2, 3]]), 
      "mass": np.array([[1, 2, 1]]),
      "acceleration": np.array([[1, 1, 3]]), 
      "time": np.array([[1, 2, 3]])}, 
     
     {"force": np.array([[-1, 2, 3]]), 
      "mass": np.array([[1, 2, 1]]),
      "acceleration": np.array([[-1, 1, 3]]), 
      "time": np.array([[1, 2, 3]])}]

units_ = {
    "force": units.Dimension("mass") * units.Dimension("length") / units.Dimension("time")**2,
    "mass": units.Dimension("mass"),
    "acceleration": units.Dimension("length") / units.Dimension("time")**2,
    "time": units.Dimension("mass") / units.Dimension("length") * units.Dimension("time")**2
}


nondims, cols, rows = nondim.dimensionless_vars(units_)
nondims = np.array(nondims).astype(float).squeeze()
print(nondims)
non_dim = []
for i, batch in enumerate(x):
    # make data into shape (1, vars, batch, dim) and nondims into shape (nondim, vars, 1, 1), then broadcast and product out the vars dimension.
    # FIXME: remove the string "elevation" from the below.
    data = np.expand_dims(np.array([batch[key] for key in cols]), 0)
    print(data, np.expand_dims(nondims, (2,3)))
    non_dim.append( (data ** np.expand_dims(nondims, (2,3))).prod(axis=1) )

non_dim


[[-1.  1.  1.  0.]
 [ 1. -2.  0.  1.]]
[[[[1 2 3]]

  [[1 2 1]]

  [[1 1 3]]

  [[1 2 3]]]] [[[[-1.]]

  [[ 1.]]

  [[ 1.]]

  [[ 0.]]]


 [[[ 1.]]

  [[-2.]]

  [[ 0.]]

  [[ 1.]]]]
[[[[-1  2  3]]

  [[ 1  2  1]]

  [[-1  1  3]]

  [[ 1  2  3]]]] [[[[-1.]]

  [[ 1.]]

  [[ 1.]]

  [[ 0.]]]


 [[[ 1.]]

  [[-2.]]

  [[ 0.]]

  [[ 1.]]]]


[array([[[1., 1., 1.]],
 
        [[1., 1., 9.]]]),
 array([[[ 1.,  1.,  1.]],
 
        [[-1.,  1.,  9.]]])]